In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"
k = 2

In [2]:
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI
from langchain.graphs import Neo4jGraph

In [3]:
graph = Neo4jGraph(
    url="bolt://localhost:7687", username="neo4j", password="$$1234bala"
)

In [6]:
from langchain.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:
# How many people played in Top Gun?
MATCH (m:Movie {{title:"Top Gun"}})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [7]:
chain.run("Who played in Top Gun?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title:"Top Gun"})<-[:ACTED_IN]-(a:Actor)
RETURN a.name AS actorName
Full Context:
[]

> Finished chain.


'Tom Cruise played the lead role in the movie Top Gun.'

# Use separate LLMs for Cypher and answer generation

In [8]:
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-4"),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    verbose=True,
    top_k=k
)

In [9]:
chain.run("Name a 2016 movie in which Mark Ruffalo, Rachel McAdams, and Michael Keaton portrayed journalists investigating a real-life scandal")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:FEATURES_ACTOR]->(a:Actor), (m:Movie)-[:RELEASED_IN]->(y:Year)
WHERE a.name IN ['Mark Ruffalo', 'Rachel McAdams', 'Michael Keaton'] AND y.year = '2016'
RETURN m.title
Full Context:
[{'m.title': 'Doctor Strange'}, {'m.title': 'Now You See Me 2'}]

> Finished chain.


'Spotlight'

In [10]:
chain.run("In which year did a science fiction thriller come out, featuring Domhnall Gleeson and Alicia Vikander, with themes of artificial intelligence and consciousness?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:BELONGS_TO_GENRE]->(g:Genre), (m:Movie)-[:FEATURES_ACTOR]->(a1:Actor), (m:Movie)-[:FEATURES_ACTOR]->(a2:Actor), (m:Movie)-[:RELEASED_IN]->(y:Year)
WHERE g.name = 'Science Fiction' AND a1.name = 'Domhnall Gleeson' AND a2.name = 'Alicia Vikander' AND m.description CONTAINS 'artificial intelligence' AND m.description CONTAINS 'consciousness'
RETURN y.year
Full Context:
[]

> Finished chain.


"The science fiction thriller featuring Domhnall Gleeson and Alicia Vikander, exploring themes of artificial intelligence and consciousness, was released in the year [unknown]. I apologize, but I don't have the specific information regarding the release year."

In [11]:
chain.run("Can you mention a 2014 release that featured Chris Pratt and Zoe Saldana as part of a team of unlikely heroes in a space opera?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:FEATURES_ACTOR]->(a:Actor), (m:Movie)-[:RELEASED_IN]->(y:Year)
WHERE a.name IN ['Chris Pratt', 'Zoe Saldana'] AND y.year = '2014' AND m.description CONTAINS 'unlikely heroes' AND m.description CONTAINS 'space opera'
RETURN m.title
Full Context:
[]

> Finished chain.


'Yes, one 2014 release that featured Chris Pratt and Zoe Saldana as part of a team of unlikely heroes in a space opera is the movie "Guardians of the Galaxy."'